In [8]:
!pip install bert_score
!pip install accelerate -U

import torch
import pandas as pd
import numpy as np
from transformers import T5Tokenizer, T5ForConditionalGeneration

test_df = pd.read_csv("test_processed.csv")

In [9]:
tokenizer_t5 = T5Tokenizer.from_pretrained("google-t5/t5-base")

device = torch.device("cuda") # GPU usage
model_t5 = T5ForConditionalGeneration.from_pretrained("google-t5/t5-base")
model_t5.to(device)

model_t5

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


T5ForConditionalGeneration(
  (shared): Embedding(32128, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=768, out_features=3072, bias=False)
              (wo): Linear(in_features=3072, out_features=768, bias=False)
              (dropout): Dro

In [10]:
MAX_ALLOWED_SEQUENCE = 8192

In [11]:
def GoBackReadingWindow(model, tokenizer, text):
    torch.cuda.empty_cache()
    if len(text.split()) < MAX_ALLOWED_SEQUENCE:
        inputs = tokenizer("summarize: " + text,\
                           return_tensors="pt",\
                           max_length=MAX_ALLOWED_SEQUENCE,\
                           truncation=True).to(device)
        outputs = model.generate(**inputs, min_length=0, max_new_tokens=MAX_ALLOWED_SEQUENCE)
        return tokenizer.decode(outputs[0], skip_special_tokens=True)

    OVERLAP = int(MAX_ALLOWED_SEQUENCE * 0.2)
    words = text.split()
    chunks = []
    start = 0

    while start < len(words):
        end = min(start + MAX_ALLOWED_SEQUENCE, len(words))
        chunks.append(' '.join(words[start:end]))
        start += MAX_ALLOWED_SEQUENCE - OVERLAP

    print(len(chunks))

    summarized_chunks = []
    for chunk in chunks:
        inputs = tokenizer("summarize: " + chunk,\
                           return_tensors="pt",\
                           max_length=MAX_ALLOWED_SEQUENCE,\
                           truncation=True).to(device)
        outputs = model.generate(**inputs, min_length=0, max_new_tokens=MAX_ALLOWED_SEQUENCE)
        summarized_chunks.append(tokenizer.decode(outputs[0], skip_special_tokens=True))
        torch.cuda.empty_cache()

    summarized_text = ' '.join(summarized_chunks)

    if len(summarized_text.split()) > MAX_ALLOWED_SEQUENCE:
        return GoBackReadingWindow(model, tokenizer, summarized_text)
    else:
        inputs = tokenizer("summarize: " + summarized_text,\
                           return_tensors="pt",\
                           max_length=MAX_ALLOWED_SEQUENCE,\
                           truncation=True).to(device)
        outputs = model.generate(**inputs, min_length=0, max_new_tokens=MAX_ALLOWED_SEQUENCE)
        return tokenizer.decode(outputs[0], skip_special_tokens=True)

In [12]:
def ReadingAfterwardsWindow(model, tokenizer, text):
    torch.cuda.empty_cache()
    if len(text.split()) < MAX_ALLOWED_SEQUENCE:
        inputs = tokenizer("summarize: " + text,\
                           return_tensors="pt",\
                           max_length=MAX_ALLOWED_SEQUENCE,\
                           truncation=True).to(device)
        outputs = model.generate(**inputs, min_length=0, max_new_tokens=MAX_ALLOWED_SEQUENCE)
        return tokenizer.decode(outputs[0], skip_special_tokens=True)

    OVERLAP = int(MAX_ALLOWED_SEQUENCE * 0.2)
    words = text.split()
    chunks = []
    start = 0

    isfirts = True
    while start < len(words):
        end = min(start + MAX_ALLOWED_SEQUENCE, len(words))
        if isfirts:
            chunks.append(' '.join(words[start:end]))
            isfirts = False
        else:
            chunks.append(' '.join(words[start + OVERLAP:end]) + " " + ' '.join(words[start:start + OVERLAP]))
        start += MAX_ALLOWED_SEQUENCE - OVERLAP

    print(len(chunks))

    summarized_chunks = []
    for chunk in chunks:
        inputs = tokenizer("summarize: " + chunk,\
                           return_tensors="pt",\
                           max_length=MAX_ALLOWED_SEQUENCE,\
                           truncation=True).to(device)
        outputs = model.generate(**inputs, min_length=0, max_new_tokens=MAX_ALLOWED_SEQUENCE)
        summarized_chunks.append(tokenizer.decode(outputs[0], skip_special_tokens=True))
        torch.cuda.empty_cache()

    summarized_text = ' '.join(summarized_chunks)

    if len(summarized_text.split()) > MAX_ALLOWED_SEQUENCE:
        return ReadingAfterwardsWindow(model, tokenizer, summarized_text)
    else:
        inputs = tokenizer("summarize: " + summarized_text,\
                           return_tensors="pt",\
                           max_length=MAX_ALLOWED_SEQUENCE,\
                           truncation=True).to(device)
        outputs = model.generate(**inputs, min_length=0, max_new_tokens=MAX_ALLOWED_SEQUENCE)
        return tokenizer.decode(outputs[0], skip_special_tokens=True)

In [13]:
def ReadingWithJumpingWindow(model, tokenizer, text):
    torch.cuda.empty_cache()
    if len(text.split()) < MAX_ALLOWED_SEQUENCE:
        inputs = tokenizer("summarize: " + text,\
                           return_tensors="pt",\
                           max_length=MAX_ALLOWED_SEQUENCE,\
                           truncation=True).to(device)
        outputs = model.generate(**inputs, min_length=0, max_new_tokens=MAX_ALLOWED_SEQUENCE)
        return tokenizer.decode(outputs[0], skip_special_tokens=True)

    JUMP = 100
    words = text.split()
    chunks = []
    start = 0

    while start < len(words):
        end = min(int(start + MAX_ALLOWED_SEQUENCE + JUMP), len(words))
        chunks.append(' '.join(words[int(start):int(start + MAX_ALLOWED_SEQUENCE/2)]) \
                        + " " + ' '.join(words[int(start + MAX_ALLOWED_SEQUENCE/2) + JUMP:end]))
        start += MAX_ALLOWED_SEQUENCE/2

    print(len(chunks))

    summarized_chunks = []
    for chunk in chunks:
        inputs = tokenizer("summarize: " + chunk,\
                           return_tensors="pt",\
                           max_length=MAX_ALLOWED_SEQUENCE,\
                           truncation=True).to(device)
        outputs = model.generate(**inputs, min_length=0, max_new_tokens=MAX_ALLOWED_SEQUENCE)
        summarized_chunks.append(tokenizer.decode(outputs[0], skip_special_tokens=True))
        torch.cuda.empty_cache()

    summarized_text = ' '.join(summarized_chunks)

    if len(summarized_text.split()) > MAX_ALLOWED_SEQUENCE:
        return ReadingWithJumpingWindow(model, tokenizer, summarized_text)
    else:
        inputs = tokenizer("summarize: " + summarized_text,\
                           return_tensors="pt",\
                           max_length=MAX_ALLOWED_SEQUENCE,\
                           truncation=True).to(device)
        outputs = model.generate(**inputs, min_length=0, max_new_tokens=MAX_ALLOWED_SEQUENCE)
        return tokenizer.decode(outputs[0], skip_special_tokens=True)

In [14]:
from bert_score import score

F1_t51 = []

for index, row in test_df.iterrows():
    reference_text = row["reference"]
    reference_summary = row["summary"]
    print(row["reference_tokens_preprocessed"])

    result_summary = GoBackReadingWindow(model_t5, tokenizer_t5, reference_text)
    print(result_summary)
    P, R, F1 = score([result_summary], [reference_summary], lang='en', verbose=False)
    print(f"T5 BertScore F1: {F1.item():.2f}")
    F1_t51.append(F1.item())
    torch.cuda.empty_cache()

np.save('F1_t51.npy', F1_t51)

sum1 = 0
for _ in F1_t51:
    sum1 += _
print(sum1 / len(F1_t51))

4996
eu regulation establishes a common principle concerning fee and charge. it also provides for the establishment of a central database on cross-border marketing. the regulation shall be binding in it entirety and directly applicable in all member states.


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


T5 BertScore F1: 0.81
14373
3
market surveillance authority may take action to reduce risk. commission may also adopt a new'reporting system' to ensure that the product is not 'in conformity' with the law.


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


T5 BertScore F1: 0.80
2073
'ancillary online service' means a service that provides a retransmission service. it also provides for the retransmission of television and radio programme. the directive shall be referred to as 'direct injection' by the commission.


...

T5 BertScore F1: 0.81
4077
a certificate granted under this regulation shall be valid for a period of six months. the certificate shall be valid for any product protected by a patent. the certificate shall be valid for any product placed on the market. a certificate granted under this regulation shall be valid for a period of five years.


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


T5 BertScore F1: 0.81
16042
3
court of justice of the european community shall be able to issue a document. document shall be accompanied by a translation into english and french. court of appeal may be set up in greece, slovakia, hungary, slovakia, poland and ireland.


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


T5 BertScore F1: 0.78
0.8031290179905407


In [15]:
F1_t52 = []

for index, row in test_df.iterrows():
    reference_text = row["reference"]
    reference_summary = row["summary"]
    print(row["reference_tokens_preprocessed"])

    result_summary = ReadingAfterwardsWindow(model_t5, tokenizer_t5, reference_text)
    print(result_summary)
    P, R, F1 = score([result_summary], [reference_summary], lang='en', verbose=False)
    print(f"T5 BertScore F1: {F1.item():.2f}")
    F1_t52.append(F1.item())
    torch.cuda.empty_cache()

np.save('F1_t52.npy', F1_t52)

sum2 = 0
for _ in F1_t52:
    sum2 += _
print(sum2 / len(F1_t52))

4996
eu regulation establishes a common principle concerning fee and charge. it also provides for the establishment of a central database on cross-border marketing. the regulation shall be binding in it entirety and directly applicable in all member states.


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


T5 BertScore F1: 0.81
14373
3
market surveillance authority shall take appropriate action to reduce risk. if a product is not complying with the regulation, it will be recalled or withdrawn. the commission may be able to provide information to the public.


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


T5 BertScore F1: 0.81
2073
'ancillary online service' means a service that provides a retransmission service. it also provides for the retransmission of television and radio programme. the directive shall be referred to as 'direct injection' by the commission.


...

T5 BertScore F1: 0.81
4077
a certificate granted under this regulation shall be valid for a period of six months. the certificate shall be valid for any product protected by a patent. the certificate shall be valid for any product placed on the market. a certificate granted under this regulation shall be valid for a period of five years.


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


T5 BertScore F1: 0.81
16042
3
court of a state bound by this convention may be sued in a civil or commercial matter. a court of justice of the european community may grant a contract to a party to this convention. a court of appeal may be set up in ireland, slovakia, poland, slovakia, greece, slovakia, hungary.


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


T5 BertScore F1: 0.79
0.8033472914746739


In [16]:
F1_t53 = []

for index, row in test_df.iterrows():
    reference_text = row["reference"]
    reference_summary = row["summary"]
    print(row["reference_tokens_preprocessed"])

    result_summary = ReadingWithJumpingWindow(model_t5, tokenizer_t5, reference_text)
    print(result_summary)
    P, R, F1 = score([result_summary], [reference_summary], lang='en', verbose=False)
    print(f"T5 BertScore F1: {F1.item():.2f}")
    F1_t53.append(F1.item())
    torch.cuda.empty_cache()

np.save('F1_t53.npy', F1_t53)

sum3 = 0
for _ in F1_t53:
    sum3 += _
print(sum3 / len(F1_t53))

4996
eu regulation establishes a common principle concerning fee and charge. it also provides for the establishment of a central database on cross-border marketing. the regulation shall be binding in it entirety and directly applicable in all member states.


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


T5 BertScore F1: 0.81
14373
4
market surveillance authority shall take appropriate action to reduce risk. if a product is not complying with the requirements of the regulation, it will be recalled. market surveillance authority shall be able to provide information on product. they may also be able to provide information on product safety and compliance.


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


T5 BertScore F1: 0.81
2073
'ancillary online service' means a service that provides a retransmission service. it also provides for the retransmission of television and radio programme. the directive shall be referred to as 'direct injection' by the commission.


...

T5 BertScore F1: 0.81
4077
a certificate granted under this regulation shall be valid for a period of six months. the certificate shall be valid for any product protected by a patent. the certificate shall be valid for any product placed on the market. a certificate granted under this regulation shall be valid for a period of five years.


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


T5 BertScore F1: 0.81
16042
4
court of a state bound by this convention may be sued in a civil or commercial matter. standing committee of experts shall be set up to examine application of the convention.


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


T5 BertScore F1: 0.79
0.8036124776391422
